In [ ]:
import backtrader as bt
import pandas as pd
import yfinance as yf

from utils.strategies import VWAPStrategy
from utils.tester import log_results, test_strategy

## Download Data from YF

In [ ]:
# Download SPX500 data from Yahoo Finance
yf_params = {'start': '2018-06-01', 'end': '2025-01-01',
             'interval': '1d', 'multi_level_index': False, 'auto_adjust': True}
data_names = [
    # '^RUT', '^IXIC', '^VIX',
    '^DJI', '^GSPC', 'GOOG', 'MSFT', 'AMZN', 'TSLA', 
              'NFLX', 'NVDA', 'AMD', 'KO']

datas = dict()
for name in data_names:
    datas[name] = yf.download(tickers=name, **yf_params)

dt_col = 'Datetime' if yf_params['interval'] == '4h' else 'Date'

for name, data in datas.items():
    data.reset_index(inplace=True)
    data[dt_col] = pd.to_datetime(data[dt_col])
    print(f"{name}: from {data[dt_col].iloc[0]} to {data[dt_col].iloc[-1]}")
data_feeds = dict()
for name, data in datas.items():
    data_feeds[name] = bt.feeds.pandafeed.PandasData(
        name=name,
        dataname=data,
        datetime=0,
        openinterest=-1,
        open=1,
        high=2,
        low=3,
        close=4,
        volume=5,
    )

## Source data locally

In [ ]:
data_local = pd.read_csv("../data/xau_usd_5m.csv", index_col=None)
data_local['datetime'] = pd.to_datetime(data_local['datetime'])
print(data_local.head())

data_feed_local = bt.feeds.PandasData(dataname=data_local, name="xau",
                                      datetime=0, openinterest=-1,
                                      open=1,
                                      high=2,
                                      low=3,
                                      close=4,
                                      volume=5)

## Backtest

In [ ]:
strategy_params = dict(
                       rrr=2.5,
)
results = dict()

for name, data_feed in data_feeds.items():
    results[name], cereb = test_strategy(VWAPStrategy, data_feed, strategy_params)
    # cereb.plot(iplot=True)

In [ ]:
for name, res in results.items():
    log_results(name, res)